In [22]:
import cv2
import numpy as np
import random
from matplotlib import pyplot as plt
from snake.imagering import trasitionExtend
from components.layer import Layer
from components.network import Network
from components.activations import logsig, satlins

In [2]:
random.seed(5219)

In [3]:
%matplotlib inline

def printBoard(board, i = 0):
    filename = "video/" + str(i).rjust(10, "0") + ".jpg"
    plt.ioff()
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.imshow(board, cmap='gray', vmin=0, vmax=1)
    plt.savefig(filename, bbox_inches='tight')
    plt.close()
    # plt.show()
    # im = Image.fromarray(board)
    # im = im.convert('RGB')
    # save_filename=filename
    # im.save(save_filename, "JPEG")

In [56]:
class Snake:
    LEFT, RIGHT, UP, DOWN = [(0,1), (0,-1), (-1,0), (1,0)]
    MAX_DIE = 25

    def __init__(self, brain) -> None:
        self._body = [(24, 24)]
        self._dir = self.LEFT
        self.total_moves = 0
        self.total_grows = 0
        self.brain = brain
        self.hist = []
        self.die = self.MAX_DIE
        self.last_dir = (0,0)
    
    def move(self, grow=False) -> bool: # return if had collision
        head = self._body[0] # head positions
        newHead = (head[0] + self._dir[0], head[1] + self._dir[1])
        collself = (self.last_dir[0] + self._dir[0], head[1] + self._dir[1])

        if self.die == 0:
            return True
        if(collself == 0):
            return True
        if ( newHead[0] < 0 or newHead [0] > 49):
            return True # collision
        if ( newHead[1] < 0 or newHead [1] > 49):
            return True # collision
        
        for part in self._body:
            if(newHead == part):
                return True # collision
            
        self.last_dir = self._dir
        
        self._body.insert(0, newHead)
        self.total_moves += 1
        if grow:
            self.die = self.MAX_DIE
            self.total_grows += 1
        else:
            self._body.pop()
            self.die -= 1
        return False
    
    def think(self, board):
        block_distances = []
        food_distances = []

        for i in range(-1, 2):
            for j in range(-1, 2):
                if j == i == 0:
                    continue
                block_distances.append(self._calc_dis(i, j, 1, board)) 
                food_distances.append(self._calc_dis(i, j, 0.5, board, True))

        tail_dir = self._dir
        size = len(self._body)
        if(size > 1):
            prev = self._body[-2]
            tail = self._body[-1]
            tail_dir = ( prev[0] - tail[0], prev[1] - tail[1] )
        
        food_distances = food_distances / (np.linalg.norm(food_distances) + 1)
        block_distances = block_distances / (np.linalg.norm(block_distances) + 1)
        input = [*food_distances, *block_distances, *self._dir, *tail_dir, size ]
        # input = [*food_distances, 0,0,0,0,0,0,0,0, 0, 0, 0,0, 0 ]
        input = input / np.linalg.norm(input)
        output = self.brain.activate(input)
        index = 0
        max = output[0]
        for i in range(1, 4):
            if (output[i] > max):
                max = output[i]
                index = i
        if index == 0:
            self._dir = self.LEFT
        elif index == 1:
            self._dir = self.RIGHT
        elif index == 2:
            self._dir = self.UP
        else:
            self._dir = self.DOWN
    
    def _calc_dis(self, x, y, goal, board, hundredIfNotFound = False):
        i, j = self.getHead()
        dist = 0
        found = False

        while (i < 49 and j < 49 and i > 0 and j > 0):
            i += x
            j += y
            dist += 1
            if(board[i][j] == goal):
                found = True
                break
        if found or not hundredIfNotFound: 
            return dist
        return -1
        
        
    def setDir(self, dir):
        sum = self._dir[0] + dir[0] + self._dir[1] + dir[1]
        if sum != 0:
            self._dir = dir
    
    def getHead(self):
        return self._body[0]
    
    def print(self, board):
        for part in self._body:
            board[part[0], part[1]] = 1
    
    def get_size(self):
        return len(self._body)
    
    def get_factor(self):
        return self.get_size() / float(self.total_moves)
    
    def copy(self):
        newLayers = []

        for layer in self.brain._layers:
            matrix = layer.get_weights().copy()
            biases = layer.get_biases().copy()
            newLayers.append(Layer(weights=matrix, biases=biases, activations=layer._activation))

        newBrain = Network(newLayers)
        return Snake(newBrain)
    
    def copy_mutate(self):
        mutation_rate = 0.5
        # copies = [m.get_weights().copy() for m in self.brain._layers]
        newLayers = []

        for layer in self.brain._layers:
            matrix = layer.get_weights().copy()
            biases = layer.get_biases().copy()
            probs = np.random.random(matrix.size).tolist()
            for idx, x in np.ndenumerate(matrix):
                if(probs[0] < mutation_rate):
                    matrix[idx] = random.random()
                    # biases[idx[0]] = random.random()
                probs.pop(0)
            newLayers.append(Layer(weights=matrix, biases=biases, activations=layer._activation))

        newBrain = Network(newLayers)
        return Snake(newBrain)

In [ ]:
board = np.zeros((50,50),dtype=np.float32)
collision = False
snake = Snake()
steps = []

while not collision:
    board.fill(0)
    collision = snake.move()
    snake.print(board)
    steps.append(board.copy())

for i in range(len(steps)):
    printBoard(steps[i], i)
        
# ffmpeg -framerate 30 -pattern_type glob -i "*.jpg" output.avi

### Brain 1
- 8 distances food in the direction (negative if there insn't)
- 8 distances to obstruction
- 2 current head direction
- 2 current tail direction
- 1 current size

i:21x1 -> h:21x21 -> h:14x21 -> o:4x14 -> 4x1

In [5]:
def brain1():
    return Network([
        Layer(
            weights = (np.random.rand(21, 21) - 0.5),
            # biases = np.random.rand(21, 1),
            biases = np.zeros(21),
            activations=logsig
        ),
        Layer(
            weights = (np.random.rand(21, 21) - 0.5),
            biases = np.zeros(21),
            activations=logsig
        ),
        Layer(
            weights = (np.random.rand(4, 21)  - 0.5),
            biases = np.zeros(4),
            activations=logsig
        )
    ])

In [58]:
def crossover(m1, m2):
    child = np.array(m1, copy=True)
    half_row = int(child.shape[0] / 2)
    child[half_row:] = m2[half_row:]
    return child

# m1 = np.ones((4,4))
# m2 = np.zeros((4,4))
# child = crossover(m1, m2)
# child

In [53]:
def rand_pos(board):
    available = []
    for idx, x in np.ndenumerate(board):
        if(board[idx] == 0):
            available.append(idx)
    idx = random.randint(0, len(available) - 1)
    return available[idx]


GEN_SIZE = 100

def execute_gen(gen):
    for i in range(GEN_SIZE):
        board = np.zeros((50,50),dtype=np.float32)
        collision = False
        snake = gen[i]
        eaten_food = False

        board[24,24] = 1
        food_pos = rand_pos(board)

        while not collision:
            board.fill(0)
            board[food_pos] = 0.5
            snake.think(board)
            collision = snake.move(grow=eaten_food)
            snake.print(board)
            eaten_food = food_pos == snake.getHead()
            if eaten_food:
                food_pos = rand_pos(board)
                eaten_food = False
            snake.hist.append(board.copy())
# for i in range(len(snake.hist)):
#     printBoard(snake.hist[i], i)

In [54]:
gen = [Snake(brain1()) for i in range(GEN_SIZE)]
bests = []

for i in range(20):
    execute_gen(gen)
    gen.sort(key=lambda x: (x.total_moves + 2 * x.total_grows), reverse=True)
    bests.append(gen[0])
    newGen  = []
    for snake in gen[:5]:
        newGen.append(snake.copy())
        for j in range(19):
            newGen.append(snake.copy_mutate())
    gen = newGen


snake = bests[-1]
# len(snake.hist)
[len(sn.hist) for sn in bests]

[41,
 27,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41]

In [55]:
for i in range(len(snake.hist)):
    printBoard(snake.hist[i], i)